# Лабораторная работа №4. Эллиптические кривые
Выполнил Ширяев Никита Алексеевич М8О-308Б-22

## Задание
Подобрать такую эллиптическую кривую, порядок точки которой полным перебором находится за 10 минут на ПК. Упомянуть в отчёте результаты замеров работы программы, характеристики вычислителя. Также указать какие алгоритмы и/или теоремы существуют для облегчения и ускорения решения задачи полного перебора.
Рассмотреть для случая конечного простого поля $\mathbb{Z}_p$.

## Характеристики компьютера
**Процессор:** Intel® Core™ i9-13900H Processor 2.6 GHz (24MB Cache, up to 5.4 GHz, 14 cores, 20 Threads)  
**Оперативная память:** 32GB LPDDR5 on board  
**Видеопамять:**
- NVIDIA® GeForce RTX™ 4060 Laptop GPU (233 AI TOPs) 8GB GDDR6
- Intel® Iris Xe Graphics

In [1]:
import random
import time

from sympy import isprime


def generate_curve_parameters(p):
    """Генерация параметров кривой, гарантирующая невырожденность"""
    while True:
        A = random.randint(1, p - 1)
        B = random.randint(1, p - 1)
        # Проверка на невырожденность: 4A³ + 27B² ≠ 0 mod p
        if (4 * pow(A, 3, p) + 27 * pow(B, 2, p)) % p != 0:
            return A, B


def find_points_on_curve(p, A, B):
    """Оптимизированный поиск точек на эллиптической кривой"""
    points = [(0, 0)]  # Нулевая точка
    quadratic_residues = {pow(y, 2, p) for y in range(p)}

    for x in range(p):
        # Вычисляем y² = x³ + Ax + B mod p
        y_squared = (pow(x, 3, p) + A * x + B) % p

        if y_squared in quadratic_residues:
            # Решение y² ≡ y_squared mod p (работает для p ≡ 3 mod 4)
            y = pow(y_squared, (p + 1) // 4, p)
            points.append((x, y))
            if y != 0:
                points.append((x, (-y) % p))

    return points


def optimized_add_points(p1, p2, p, A):
    """Оптимизированное сложение точек на эллиптической кривой"""
    if p1 == (0, 0):
        return p2
    if p2 == (0, 0):
        return p1

    x1, y1 = p1
    x2, y2 = p2

    if x1 == x2 and (y1 + y2) % p == 0:
        return (0, 0)  # Точка на бесконечности

    if p1 == p2:
        # Удвоение точки
        inv_denominator = pow(2 * y1, p - 2, p)  # Обратный элемент
        m = (3 * pow(x1, 2, p) + A) * inv_denominator % p
    else:
        # Сложение разных точек
        inv_denominator = pow(x2 - x1, p - 2, p)
        m = (y2 - y1) * inv_denominator % p

    x3 = (pow(m, 2, p) - x1 - x2) % p
    y3 = (m * (x1 - x3) - y1) % p

    return (x3, y3)


def compute_point_order(point, p, A):
    """Вычисление порядка точки полным перебором"""
    current_point = point
    order = 1

    while current_point != (0, 0):
        current_point = optimized_add_points(current_point, point, p, A)
        order += 1

    return order


def find_optimal_curve(
    target_time=600, tolerance=60, start_p=15 * 10**7, end_p=15 * 10**7 + 1000
):
    """Поиск кривой с временем вычисления порядка точки ≈ target_time"""
    # Генерируем список простых чисел в заданном диапазоне
    primes = [p for p in range(start_p, end_p + 1) if isprime(p)]
    results = []
    for p in primes:
        # Генерация параметров кривой
        A, B = generate_curve_parameters(p)

        # Поиск точек на кривой
        start_time = time.time()
        points = find_points_on_curve(p, A, B)

        if len(points) < 5:  # Пропускаем кривые с малым числом точек
            continue

        # Выбираем случайную точку (исключая нулевую)
        point = random.choice(points[1:])

        # Вычисляем порядок точки
        order = compute_point_order(point, p, A)
        elapsed = time.time() - start_time

        results.append((p, elapsed, order))

        print(f"p = {p}, Time = {elapsed:.2f}s, Order = {order}")

        # Проверяем, попадает ли время в целевой диапазон
        if abs(elapsed - target_time) <= tolerance:
            print("\nНайдена подходящая кривая!")
            print(f"y² ≡ x³ + {A}x + {B} mod {p}")
            print(f"Поле: Z_{p}")
            print(f"Точка: {point}, Порядок: {order}")
            print(f"Всего точек на кривой: {len(points)}")
            print(f"Время вычисления: {elapsed:.2f} секунд")
            break
    else:
        # Если не нашли точного соответствия, берем ближайший вариант
        closest = min(results, key=lambda x: abs(x[1] - target_time))
        p, elapsed, order = closest
        print(f"\nБлижайший результат: p = {p}, Time = {elapsed:.2f}s")


find_optimal_curve(target_time=600, tolerance=60)

p = 150000001, Time = 510.68s, Order = 74992565
p = 150000029, Time = 687.66s, Order = 149977074
p = 150000047, Time = 563.73s, Order = 75009072

Найдена подходящая кривая!
y² ≡ x³ + 92733659x + 12899758 mod 150000047
Поле: Z_150000047
Точка: (10211307, 119741910), Порядок: 75009072
Всего точек на кривой: 150018144
Время вычисления: 563.73 секунд


## Алгоритмы и теоремы для ускорения полного перебора

### 1. Теорема Хассе
   - Позволяет оценить количество точек на эллиптической кривой над конечным полем $\mathbb{Z}_p$:
   $$
     |p + 1 - \#E(\mathbb{Z}_p)| \leq 2\sqrt{p}
   $$
   - Это означает, что порядок группы точек кривой $\#E(\mathbb{Z}_p)$ лежит в интервале $[p + 1 - 2\sqrt{p}, p + 1 + 2\sqrt{p}]$.
   - Позволяет сократить перебор при вычислении порядка группы точек.

### 2. Алгоритм Шуфа (Schoof's Algorithm)
   - Эффективный алгоритм для вычисления точного количества точек на эллиптической кривой над конечным полем.
   - Основан на свойствах деления полиномов и модулярной арифметике.
   - Позволяет точно вычислить $\#E(\mathbb{Z}_p)$ за полиномиальное время.

### 3. Алгоритм Baby-step Giant-step
   - Оптимизированный метод для нахождения порядка точки.
   - Основан на идее разделения задачи на две части: "маленькие шаги" и "большие шаги".
   - Сложность $O(\sqrt{n})$, где $n$ — порядок точки.

### 4. Метод Полларда (Pollard's Rho)
   - Вероятностный алгоритм для решения задачи дискретного логарифмирования.
   - Может быть адаптирован для нахождения порядка точки.
   - Эффективен для больших групп.

### 5. Оптимизация сложения точек
   - Использование проективных координат вместо аффинных для исключения дорогих операций деления.
   - Формулы сложения и удвоения точек можно оптимизировать для конкретных параметров кривой.

### 6. Теорема Лагранжа
   - Порядок любой точки делит порядок группы.
   - После вычисления $\#E(\mathbb{Z}_p)$ можно проверять только делители этого числа как возможные порядки точки.